# prepare text and audio_paths files

## preprocess audio 

In [ ]:
import os
import pandas as pd
import random

seed = 17
random.seed(seed)

data_dir = "data/midea_2173"   # your midea data directory

df_trans = pd.read_csv(os.path.join(data_dir, 'transcripts.csv'))

print(df_trans.shape)
df_trans.rename(columns={'content': 'sentence', 'record_file_id': 'file_id'}, inplace=True)
df_trans.head()

## convert amr to wav

In [ ]:
!pip install pydub

In [ ]:
from pydub import AudioSegment
from tqdm import tqdm

for i, row in tqdm(df_trans.iterrows(), total=df_trans.shape[0]):
    filename = row['file_id'][:-3]
    amr_audio = AudioSegment.from_file(data_dir+f'/amrs/{filename}amr', format="amr")
    amr_audio.export(data_dir+f'/wavs/{filename}wav', format="wav")
    df_trans.loc[i, 'file_id'] = f'{filename}wav'

df_trans.to_csv(data_dir+'/transcripts.csv', index=False)
df_trans.head()

## combine different datasets to form a custom dataset

In [ ]:
# custom_dir = 'data/custom_data_v1' # 2k mdcc, 2k cmcc, 318 midea
# custom_dir = 'data/custom_data_v2' # 65120 mdcc, 8429 cmcc, 300 midea
# custom_dir = 'data/custom_data_v3' # 65120 mdcc, 8429 cmcc
# custom_dir = 'data/custom_data_v4' # 5k mdcc, 5k cmcc
# custom_dir = 'data/custom_data_v5' # 2k mdcc, 2k cmcc
# custom_dir = 'data/custom_data_v6' # 5k mdcc, 5k cmcc, 318 midea
custom_dir = 'data/custom_data_v7' # 5k mdcc, 5k cmcc, 1303 midea
# custom_dir = 'data/custom_data_v8' # 1k mdcc, 1k cmcc, 1303 midea

os.makedirs(custom_dir, exist_ok=True)

text_path = os.path.join(custom_dir, 'text')
audio_paths_path = os.path.join(custom_dir, 'audio_paths')

### MDCC data

In [ ]:
import uuid
import shutil

mdcc_dir = 'data/MDCC'

splits = ['train', 'valid', 'test']
# counts = [2000, 200, 200]
# counts = [65120, 5663, 12492]
# counts = [65120, 5663, 12492]
# counts = [5000, 1000, 1000]
# counts = [2000, 200, 200]
# counts = [5000, 200, 200]
counts = [5000, 500, 500]
# counts = [1000, 200, 200]

for split, count in zip(splits, counts):
    save_dir = os.path.join(custom_dir, split)
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    
    df = pd.read_csv(os.path.join(mdcc_dir, f'cnt_asr_{split}_metadata.csv'))
    print(split, df.shape)
    print(df.head())
    
    df_sample = df.sample(n=min(count, df.shape[0]), random_state=seed)
    with open(os.path.join(custom_dir, split, 'text'), 'a') as fo1, open(os.path.join(custom_dir, split, 'audio_paths'), 'a') as fo2:
        for i, row in df_sample.iterrows():
            unique_id = str(uuid.uuid4())
            with open(os.path.join(mdcc_dir, row['text_path'][2:])) as fi:
                trans = fi.read()
                trans = trans.strip()

            audio_path = os.path.realpath(mdcc_dir+'/'+row['audio_path'][2:])

            # shutil.copy2(audio_src, audio_dest)
            
            # if i < df_sample.shape[0] - 1:
            fo1.write(f"{unique_id} {trans}\n")
            fo2.write(f"{unique_id} {audio_path}\n")
            # else:
            #     fo1.write(f"{unique_id} {trans}\n")
            #     fo2.write(f"{unique_id} {audio_path}\n")

## Common Voice 17.0

In [ ]:
import uuid
import shutil

common_dir = 'data/cv-corpus-17.0-2024-03-15/zh-HK'
splits = ['train', 'valid', 'test']
# counts = [2000, 200, 200]
# counts = [8429, 5595, 5595]
# counts = [8429, 5595, 5595]
# counts = [5000, 1000, 1000]
# counts = [2000, 200, 200]
# counts = [5000, 200, 200]
counts = [5000, 500, 500]
# counts = [1000, 200, 200]

for split, count in zip(splits, counts):
    save_dir = os.path.join(custom_dir, split)
    
    df = pd.read_csv(os.path.join(common_dir, f'{split}.tsv'), sep='\t')
    print(split, df.shape)
    # print(df.head())
    
    df_sample = df.sample(n=min(count, df.shape[0]), random_state=seed)
    with open(os.path.join(custom_dir, split, 'text'), 'a') as fo1, open(os.path.join(custom_dir, split, 'audio_paths'), 'a') as fo2:
        for i, row in df_sample.iterrows():
            unique_id = str(uuid.uuid4())
            trans = row['sentence']
            audio_path = os.path.realpath(common_dir+'/clips/'+row['path'])

            # shutil.copy2(audio_src, audio_dest)

            # if i < df_sample.shape[0] - 1:
            fo1.write(f"{unique_id} {trans}\n")
            fo2.write(f"{unique_id} {audio_path}\n")
            # else:
            #     fo1.write(f"{unique_id} {trans}\n")
            #     fo2.write(f"{unique_id} {audio_path}\n")

## Midea Data

In [ ]:
import uuid
import shutil

midea_dir = 'data/midea_2173'
splits = ['train', 'valid', 'test']
ratios = [0.6, 0.2, 0.2]

df = pd.read_csv(os.path.join(midea_dir, f'transcripts.csv'))
df = df.sample(frac=1, random_state=seed).reset_index(drop=True)
print(df.shape)
print(df.head())

prev_count = 0
num = df.shape[0]
for split, ratio in zip(splits, ratios):
    save_dir = os.path.join(custom_dir, split)
    
    count = int(num*ratio)
    print(split, count)

    df_sample = df.loc[prev_count:prev_count+count]
    prev_count += count
    
    with open(os.path.join(custom_dir, split, 'text'), 'a') as fo1, open(os.path.join(custom_dir, split, 'audio_paths'), 'a') as fo2:
        for i, row in df_sample.iterrows():
            unique_id = str(uuid.uuid4())
            trans = row['sentence']
            audio_path = os.path.realpath(midea_dir+'/wavs/'+row['file_id'])

            # shutil.copy2(audio_src, audio_dest)

            # if i < df_sample.shape[0] - 1:
            fo1.write(f"{unique_id} {trans}\n")
            fo2.write(f"{unique_id} {audio_path}\n")
            # else:
            #     fo1.write(f"{unique_id} {trans}")
            #     fo2.write(f"{unique_id} {audio_path}")

In [ ]:
# # remove dirs

# shutil.rmtree(custom_dir)

# process data

In [ ]:
!python3 finetune/custom_data/data_prep.py \
--source_data_dir data/${custom_dir}/test \
--output_data_dir data/${custom_dir}/test

In [ ]:
!python3 finetune/custom_data/data_prep.py \
--source_data_dir data/${custom_dir}/valid \
--output_data_dir data/${custom_dir}/valid

In [ ]:
!python3 finetune/custom_data/data_prep.py \
--source_data_dir data/${custom_dir}/train \
--output_data_dir data/${custom_dir}/train

# Try to load the processed data

In [ ]:
# import argparse
# from datasets import DatasetDict, Audio, load_from_disk, concatenate_datasets

# train_datasets = ['data/custom_data_v1/train']
# eval_datasets = ['data/custom_data_v1/valid']

# def load_custom_dataset(split):
#     ds = []
#     if split == 'train':
#         for dset in train_datasets:
#             ds.append(load_from_disk(dset))
#     if split == 'eval':
#         for dset in eval_datasets:
#             ds.append(load_from_disk(dset))

#     ds_to_return = concatenate_datasets(ds)
#     ds_to_return = ds_to_return.shuffle(seed=22)
#     return ds_to_return

# ds_to_return = load_custom_dataset('eval')
# ds_to_return

# for ex in ds_to_return:
#     print(ex)

# def prepare_dataset(batch):
#     # load and (possibly) resample audio data to 16kHz
#     audio = batch["audio"]

#     # compute log-Mel input features from input audio array 
#     batch["input_features"] = processor.feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]
#     # compute input length of audio sample in seconds
#     batch["input_length"] = len(audio["array"]) / audio["sampling_rate"]
    
#     # optional pre-processing steps
#     transcription = batch["sentence"]
#     if do_lower_case:
#         transcription = transcription.lower()
#     if do_remove_punctuation:
#         transcription = normalizer(transcription).strip()
    
#     # encode target text to label ids
#     batch["labels"] = processor.tokenizer(transcription).input_ids
#     return batch

# max_label_length = 225 # model.config.max_length
# min_input_length = 0.0
# max_input_length = 30.0
# def is_in_length_range(length, labels):
#     return min_input_length < length < max_input_length and 0 < len(labels) < max_label_length


# print('DATASET PREPARATION IN PROGRESS...')
# raw_dataset = DatasetDict()
# # raw_dataset["train"] = load_custom_dataset('train')
# raw_dataset["eval"] = load_custom_dataset('eval')

# raw_dataset = raw_dataset.cast_column("audio", Audio(sampling_rate=args.sampling_rate))
# raw_dataset = raw_dataset.map(prepare_dataset, num_proc=args.num_proc)

# raw_dataset = raw_dataset.filter(
#     is_in_length_range,
#     input_columns=["input_length", "labels"],
#     num_proc=args.num_proc,
# )

# ###############################     DATA COLLATOR AND METRIC DEFINITION     ########################

# @dataclass
# class DataCollatorSpeechSeq2SeqWithPadding:
#     processor: Any

#     def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
#         # split inputs and labels since they have to be of different lengths and need different padding methods
#         # first treat the audio inputs by simply returning torch tensors
#         input_features = [{"input_features": feature["input_features"]} for feature in features]
#         batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

#         # get the tokenized label sequences
#         label_features = [{"input_ids": feature["labels"]} for feature in features]
#         # pad the labels to max length
#         labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

#         # replace padding with -100 to ignore loss correctly
#         labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

#         # if bos token is appended in previous tokenization step,
#         # cut bos token here as it's append later anyways
#         if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
#             labels = labels[:, 1:]

#         batch["labels"] = labels

#         return batch

# data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)
# print('DATASET PREPARATION COMPLETED')

# separate different channels of wavs

In [ ]:
# # %pip install scipy

# import numpy as np
# import scipy.io.wavfile as wavfile

# # Read the multi-channel WAV file
# input_filename = 'data/midea_0612/wavs/d5f2afaa-53af-4dcb-ac24-3827a99c748e.wav'
# sample_rate, data = wavfile.read(input_filename)

# # Ensure the data is two-dimensional (i.e., multiple channels)
# if len(data.shape) == 1:
#     raise ValueError("The provided WAV file is not multi-channel.")

# # Get the number of channels
# num_channels = data.shape[1]

# # Loop through each channel and save it as a separate WAV file
# for i in range(num_channels):
#     channel_data = data[:, i]
    
#     output_filename = input_filename[:-4].replace('wavs', 'wavs_1channel')+f'_c{i}.wav'
#     wavfile.write(output_filename, sample_rate, channel_data)
#     print(f'Channel {i+1} saved as {output_filename}')


# ## another way to separate different channels

In [ ]:
# !pip install pydub

In [ ]:
from pydub import AudioSegment

def split_channels(audio_path, output_prefix):
    # Load the multi-channel audio file
    audio = AudioSegment.from_file(audio_path)

    # Get the number of channels
    channels = audio.split_to_mono()
    print(len(channels))

    # Save each channel as a separate single-channel audio file
    for i, channel in enumerate(channels):
        output_path = f"{output_prefix}_channel_{i+1}.wav"
        channel.export(output_path, format="wav")
        print(f"Saved {output_path}")

# Example usage
audio_path = 'data/midea_0612/wavs/3ee4b9f4-7674-4978-9066-d89b46c9adb4.wav'
output_prefix = "data/midea_0612/wavs_1channel/3ee4b9f4-7674-4978-9066-d89b46c9adb4"
split_channels(audio_path, output_prefix)

In [1]:
from pydub import AudioSegment
import os
from tqdm import tqdm

# Function to split audio file into chunks of given duration
def split_audio(file_path, chunk_length_ms=10000):
    audio = AudioSegment.from_wav(file_path)
    total_length_ms = len(audio)
    
    chunks = []
    for start_ms in range(0, total_length_ms, chunk_length_ms):
        end_ms = min(start_ms + chunk_length_ms, total_length_ms)
        chunk = audio[start_ms:end_ms]
        chunks.append(chunk)
    
    return chunks

# Function to save audio chunks to files and separate channels
def save_chunks_with_channels(chunks, output_dir, base_filename="chunk", chunk_length=10):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    for i, chunk in enumerate(chunks):
        # Separate channels
        left_channel = chunk.split_to_mono()[0]
        right_channel = chunk.split_to_mono()[1]
        
        # Save left and right channels separately
        left_filename = os.path.join(output_dir, f"{base_filename}_{i*chunk_length}_customer.wav")
        right_filename = os.path.join(output_dir, f"{base_filename}_{i*chunk_length}_agent.wav")
        
        left_channel.export(left_filename, format="wav")
        right_channel.export(right_filename, format="wav")
        
        # print(f"Saved chunk {i} left channel: {left_filename}")
        # print(f"Saved chunk {i} right channel: {right_filename}")

# Define the file path and output directory
# data_dir = 'data/midea_dialogue/long'
# data_dir = 'data/test_files_20240816/ai辅助填单数据/录音文件'
data_dir = 'data/test_0823'

filenames = [filename for filename in os.listdir(data_dir) if filename.endswith('wav')]
print(len(filenames), filenames[:5])

chunk_length_ms = 10000 # ms

for filename in tqdm(filenames, total=len(filenames)):
    input_file_path = os.path.join(data_dir, filename)
    output_directory = "data/test_0823/short_10s"

    # Split the audio file into 10-second chunks
    chunks = split_audio(input_file_path, chunk_length_ms=chunk_length_ms)

    basename = filename.split('.')[0]
    # Save the chunks with separate channels
    save_chunks_with_channels(chunks, output_directory, basename, chunk_length=(chunk_length_ms//1000))

1 ['14468275-223c-41d7-af78-1e5abea9a8c7_313_agent.wav']


  0%|          | 0/1 [00:00<?, ?it/s]


IndexError: list index out of range

In [4]:
!aws s3 sync data/test_0823 s3://sagemaker-us-west-2-452145973879/datasets/midea_data/test_0823/short_10s

upload: data/test_0823/14468275-223c-41d7-af78-1e5abea9a8c7_313_agent.wav to s3://sagemaker-us-west-2-452145973879/datasets/midea_data/test_0823/short_10s/14468275-223c-41d7-af78-1e5abea9a8c7_313_agent.wav
